# MHC - CH variants on all data 
- Barbara Walkowiak (bw450)

- 2024-06-19, modified 2024-09-07

- In this script, for each individual screened for CH in the UKB I add MHC binding scores based on their MHC genotype (class I)


# Set up

In [1]:
# IMPORTS

import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.patches import Polygon
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib import cm
import scipy.special
from scipy import integrate
import scipy.integrate as it
from scipy.interpolate import interp1d
from scipy.stats import kde
import copy
import glob, os
import re
# from sklearn import datasets, linear_model
import pandas as pd
from decimal import *
from operator import itemgetter    
from collections import OrderedDict
import timeit
import time 
import csv
import seaborn as sns 
import scipy as sp
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels.api as sm
import plotly.express as px
import kaleido


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# specify font for plotting 
plt.rcParams.update({'font.sans-serif':'Helvetica'})

# stop printing warnings 
import warnings
warnings.filterwarnings("ignore")

# get current date 
timestr = time.strftime("%Y%m%d") 

# Read in dataframe with imputed MHC I and CH status for each participant

In [3]:
df_hla1 = pd.read_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_df_hla1_ch_status.csv')

# Define necessary functions

In [4]:
# define functions to make the HLA names uniform

# transform format (from NetMHCpan data to UKBB format)
def transform_format_netmhc(input_string):
    
    # a regular expression pattern to match the input format
    pattern = re.compile(r'HLA-(\w\S*)(\d{2}):(\d{2})')

    # check if there is a match
    match = pattern.match(input_string)

    # if match, apply transformation
    if match:
        group1 = match.group(1).replace('*', '') # remove the star 
        group2 = int(match.group(2)) # remove zeros at the start 
        group3 = match.group(3) # leave as it is 

        # Format the output string
        output_string = f'{group1}_{group2}{group3}' # stitch back 
        return output_string # return transformed string 

    # if no much, return original string 
    return input_string

# transform format (from PRIME format to UKBB format)
def transform_format_prime(input_string):

    # regular expression pattern to match the input format
    # Nb PRIME is almost the same as NetMHC but there is no star after "HLA-A*"
    pattern = re.compile(r'HLA-(\w)(\d{2}):(\d{2})')

    # check if there is a match
    match = pattern.match(input_string)

    # if match, apply transformation
    if match:
        group1 = match.group(1).replace('*', '') 
        group2 = int(match.group(2)) 
        group3 = match.group(3) 

        # Format the output string
        output_string = f'{group1}_{group2}{group3}' # stitch back 
        return output_string # return transformed string 

    # if no match, return original string 
    return input_string

In [5]:
# for everyone, assign the score for every variant examined based on their MHC I genotype 
# define the function to find best scores (possible to do this for different parameters, we chose %Rank_EL)

def find_best_score_for_all_variants(row, df, param):

    '''
    row = function is applied to each row of the participant dataframe (ie run through each participant)

    df = dataframe with prediction scores (like netmhc)

    Allowed parameters (param) are:
    Aff_nM - affinity (raw number)
    Score_BA - binding affinity score
    Score_EL - elution score
    %Rank_BA - %Rank of binding affinity cf a set of random peptides
    %Rank_EL - %Rank of elution cf a set of random peptides
    '''

    # get HLAs for each person 
    hlas = row.index[7:][row[7:] >= 1] # select alleles which each Person (row) carries (the first 7 columns are: Person ID, gene_var, VAF, CH status, age, depth, var_depth)
   
    # get variants 
    variants = df['gene_var']
   
    scores = {} # initialise empy dictionaries

    # depending on the parameter, pick the minimum of maximum value 
    if param == "Aff_nM":
        for var in variants:
            # Find the minimum value for each variant in the category that is present
            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0])
            # Update the dictionary with the minimum value for the corresponding variant
            scores[f'score_{var}'] = best_value
        return pd.Series(scores)

    elif param == "Score_BA":
        for var in variants:
            
            best_value = max(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value
        
        return pd.Series(scores)

    elif param == "Score_EL":
        for var in variants:
           
            best_value = max(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value

        return pd.Series(scores)

    elif param == "%Rank_BA":
        for var in variants:

            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value

        return pd.Series(scores)

    # we will likely be choosing this option
    elif param == "%Rank_EL":
        for var in variants:
            
            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0]) # choose minimum rank as the best score 
            scores[f'score_{var}'] = best_value

        return pd.Series(scores)


## Load binding predictions for MHC I from NetMHC I

In [6]:

# Import NetMHC scores  
pred_file_net = '/Users/barbarawalkowiak/Desktop/msc_thesis/netMHC_out/scores/20240716_NetMHC_HLA_UKBB_with_affinities_bestscores_allvariants.csv' # EL scores, BA scores, EL rank, BA rank, affinity prediction
pred_filename_net = pred_file_net.split('/')[2].split('.')[0] # identify file name 
pred_method_net = pred_file_net.split('_out')[0] # identify method used to obtain predictions

# load the csv
netmhc = pd.read_csv(pred_file_net) # load the csv 
netmhc = netmhc.rename(columns={'Aff(nM)': 'Aff_nM'})

# specify HLA allele naming format 
NET_col = netmhc.HLA
NET_formatted = NET_col.apply(transform_format_netmhc)
netmhc = pd.concat([netmhc, NET_formatted.rename('HLA_formatted')], axis = 1)

# select required columns and sor values 
netmhc = netmhc[['HLA_formatted', 'Peptide', '%Rank_EL', 'Score_EL', '%Rank_BA', 'Score_BA', 'Aff_nM', 'gene', 'variant', 'genotype']]
netmhc = netmhc.sort_values(by=['HLA_formatted', 'gene', 'variant', 'genotype'])

# identify gene variants 
netmhc['gene_var_gt'] = netmhc['gene'] + '_' + netmhc['variant'] + '_' + netmhc['genotype'] # add complete genotype data
netmhc['gene_var'] = netmhc['gene'] + '_' + netmhc['variant']
netmhc['varID'] = netmhc['gene'] + ' ' + netmhc['variant'] # this is a column where the variant ID is in the same format as in the CH cases dataframe
netmhc = netmhc.rename(columns={'Aff(nM)': 'Aff_nM'}) # rename affinity column 
scores_netmhc = netmhc[['HLA_formatted', 'Score_EL', '%Rank_EL', 'Score_BA', '%Rank_BA', 'Aff_nM', 'varID', 'gene_var', 'gene_var_gt']] # select columns of interest

# Print the results 
print('Number of alleles for which predictions are available (NetMHC):', len(netmhc.HLA_formatted.unique()))

# Find MHC alleles which have been typed in the UKBB
hla_ukbb = df_hla1.filter(regex='\d').columns # HLA from all UKBB
print('Number of HLA alleles which have been identified in the UK BioBank:', len(hla_ukbb))

# Now look at variants
print('Number of unique variants I have predictions for (NetMHC):',  len(netmhc.gene_var.unique()))


Number of alleles for which predictions are available (NetMHC): 194
Number of HLA alleles which have been identified in the UK BioBank: 215
Number of unique variants I have predictions for (NetMHC): 54


### variants identified in CH screening vs variants we have predictions for (all variants screened for should be covered)

In [7]:
print('Number of variants identified in the UKBB from CH screening:', len(df_hla1.gene_var.unique())-1) # NB this list also includes 'NaN'
print('Number or variants I have predictions for (NetMHC):',  len(netmhc.gene_var.unique()))

variants_in_ukbb = df_hla1.gene_var.unique().tolist()
variants_in_ukbb = [x for x in variants_in_ukbb if str(x) != 'nan']
print('List of variants examined:', sorted(variants_in_ukbb))
print('List of variants with NetMHC predictions:',  sorted(netmhc.gene_var.unique())) # also includes predictions for variants which turned out not to be common enough  

Number of variants identified in the UKBB from CH screening: 40
Number or variants I have predictions for (NetMHC): 54
List of variants examined: ['DNMT3A_P904L', 'DNMT3A_P904Q', 'DNMT3A_P904R', 'DNMT3A_R320*', 'DNMT3A_R326C', 'DNMT3A_R326G', 'DNMT3A_R326S', 'DNMT3A_R598*', 'DNMT3A_R729G', 'DNMT3A_R729W', 'DNMT3A_R736C', 'DNMT3A_R736G', 'DNMT3A_R736H', 'DNMT3A_R736L', 'DNMT3A_R736S', 'DNMT3A_R771*', 'DNMT3A_R882C', 'DNMT3A_R882H', 'DNMT3A_R882L', 'DNMT3A_R882P', 'DNMT3A_R882S', 'DNMT3A_Y735C', 'DNMT3A_Y735F', 'DNMT3A_Y735S', 'GNB1_K57E', 'IDH1_R132H', 'IDH2_R140Q', 'IDH2_R172K', 'JAK2_V617F', 'KRAS_G12D', 'KRAS_G12S', 'MPL_W515L', 'NRAS_G12D', 'SF3B1_K666N', 'SF3B1_K700E', 'SRSF2_P95H', 'SRSF2_P95L', 'SRSF2_P95R', 'TP53_R175H', 'TP53_R273H']
List of variants with NetMHC predictions: ['DNMT3A_P904L', 'DNMT3A_P904Q', 'DNMT3A_P904R', 'DNMT3A_R320*', 'DNMT3A_R326C', 'DNMT3A_R326G', 'DNMT3A_R326S', 'DNMT3A_R598*', 'DNMT3A_R598G', 'DNMT3A_R729G', 'DNMT3A_R729W', 'DNMT3A_R736C', 'DNMT3A_R736G

In [8]:

# create a suitably formatted NetMHC df 
param = '%Rank_EL'
netmhc_sub = netmhc[['HLA_formatted', 'gene_var_gt', param]]
netmhc_sub_wide = pd.pivot(netmhc_sub, index='gene_var_gt', columns='HLA_formatted', values=param)
netmhc_sub_wide = netmhc_sub_wide.reset_index() # this is to make sure that you have the gene_var column in there too 
netmhc_sub_wide.columns = map(transform_format_netmhc, netmhc_sub_wide.columns)
netmhc_sub_wide


,gene_var_gt,A_101,A_102,A_103,A_1101,A_1102,A_1103,A_201,A_202,A_203,...,C_403,C_407,C_501,C_602,C_701,C_702,C_704,C_801,C_802,C_804
0,DNMT3A_P904L_ch,1.304,1.295,1.163,2.286,2.286,1.618,0.753,0.645,0.238,...,3.376,1.829,4.538,0.771,0.951,1.073,2.260,2.349,4.492,2.495
1,DNMT3A_P904L_refseq,0.968,0.884,0.844,1.602,1.602,0.793,2.147,2.519,1.271,...,1.485,0.405,2.360,0.822,0.643,0.273,0.857,1.867,2.322,2.149
2,DNMT3A_P904Q_ch,1.018,1.049,0.902,2.498,2.498,1.530,0.807,0.565,0.235,...,2.449,1.157,2.847,0.655,0.804,0.768,2.334,2.087,2.773,2.327
3,DNMT3A_P904Q_refseq,0.968,0.884,0.844,1.602,1.602,0.793,2.147,2.519,1.271,...,1.485,0.405,2.360,0.822,0.643,0.273,0.857,1.867,2.322,2.149
4,DNMT3A_P904R_ch,1.331,1.437,1.222,2.440,2.440,1.650,0.361,0.321,0.091,...,6.394,2.204,6.166,0.513,0.504,0.491,1.427,6.255,8.836,5.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,SRSF2_P95R_refseq,6.221,3.614,6.530,4.539,4.539,5.012,21.355,15.723,12.063,...,5.934,3.531,7.691,4.075,1.232,2.088,6.109,11.847,6.015,10.009
104,TP53_R175H_ch,5.285,5.159,6.238,3.879,3.879,3.415,1.761,1.447,1.270,...,11.861,9.305,13.711,3.284,2.270,3.036,2.763,9.837,13.844,7.516
105,TP53_R175H_refseq,9.617,6.019,11.430,2.145,2.145,2.412,2.504,2.000,1.706,...,13.622,10.869,14.295,2.364,1.979,4.188,3.226,12.513,16.362,9.801
106,TP53_R273H_ch,7.237,10.392,7.881,7.668,7.668,8.440,1.913,2.140,3.311,...,10.534,4.467,8.629,0.132,0.266,1.034,0.579,3.201,9.857,3.914


In [9]:

# create a list of HLA alleles genotyped in the UKBB for which predictions are available
hla_intersect = netmhc_sub_wide.columns[netmhc_sub_wide.columns.isin(hla_ukbb)] # HLA in the UKBB which I have predictions for 
hla_intersect_list = hla_intersect.tolist() # 194 alleles 

# create the prediction dataset 
netmhc_sub = netmhc_sub_wide[hla_intersect_list + netmhc_sub_wide.columns[netmhc_sub_wide.columns.str.contains('gene_var')].tolist()] # subset netmhc so you only have alleles which are in the UKBB, also 194
netmhc_sub = netmhc_sub[netmhc_sub['gene_var_gt'].str.contains('_ch', regex=True)] # retain CH scores only 
netmhc_sub['gene_var'] = netmhc_sub['gene_var_gt'].str.replace('_ch', '') # remove the ch / refseq annotation


In [10]:
# subset to only get predictions for variants which are investigated 
netmhc_sub2 = netmhc_sub[netmhc_sub['gene_var'].isin(variants_in_ukbb)]
netmhc_sub2

,A_101,A_102,A_103,A_1101,A_1102,A_1103,A_201,A_202,A_203,A_205,...,C_501,C_602,C_701,C_702,C_704,C_801,C_802,C_804,gene_var_gt,gene_var
0,1.304,1.295,1.163,2.286,2.286,1.618,0.753,0.645,0.238,0.974,...,4.538,0.771,0.951,1.073,2.260,2.349,4.492,2.495,DNMT3A_P904L_ch,DNMT3A_P904L
2,1.018,1.049,0.902,2.498,2.498,1.530,0.807,0.565,0.235,0.752,...,2.847,0.655,0.804,0.768,2.334,2.087,2.773,2.327,DNMT3A_P904Q_ch,DNMT3A_P904Q
4,1.331,1.437,1.222,2.440,2.440,1.650,0.361,0.321,0.091,0.439,...,6.166,0.513,0.504,0.491,1.427,6.255,8.836,5.360,DNMT3A_P904R_ch,DNMT3A_P904R
6,37.267,47.917,36.111,27.250,27.250,23.808,52.667,55.000,47.400,60.417,...,52.000,61.667,53.333,50.000,60.000,45.000,52.500,38.333,DNMT3A_R320*_ch,DNMT3A_R320*
8,3.837,4.932,4.312,2.104,2.104,2.983,3.226,2.774,3.044,1.188,...,1.078,1.749,0.881,1.618,3.397,0.660,0.738,1.085,DNMT3A_R326C_ch,DNMT3A_R326C
10,2.122,2.566,1.955,1.539,1.539,1.707,6.313,4.497,3.213,1.820,...,0.356,1.624,0.992,1.683,3.454,0.147,0.240,0.222,DNMT3A_R326G_ch,DNMT3A_R326G
12,2.071,2.607,1.932,0.749,0.749,0.972,2.538,1.634,1.243,0.539,...,0.362,0.784,0.346,0.699,1.781,0.138,0.248,0.233,DNMT3A_R326S_ch,DNMT3A_R326S
14,23.380,22.913,25.087,1.678,1.678,1.514,30.909,45.538,32.714,36.703,...,56.250,32.750,21.231,22.812,55.000,49.000,65.000,51.250,DNMT3A_R598*_ch,DNMT3A_R598*
18,0.811,1.196,0.766,2.261,2.261,3.622,0.167,0.085,0.035,0.242,...,3.118,4.558,1.804,0.599,1.615,3.944,2.924,3.740,DNMT3A_R729G_ch,DNMT3A_R729G
20,1.603,1.265,1.485,2.978,2.978,5.565,0.218,0.153,0.078,0.397,...,2.903,4.263,1.218,0.336,1.324,4.146,2.917,4.521,DNMT3A_R729W_ch,DNMT3A_R729W


In [11]:
# apply the function to find, for each participant, the best score for each examined CH variant 
# you take your current df and apply, row by row, the function to get scores for each variant 
# for the purpose of this, we first need to select MHC columns which are actually in the hla_intersect_list I think 
# DO NOT use this for sth like MHC heterozygosity tho!!
df_hla1_hlas = pd.concat([df_hla1[['Person_ID', 'gene_var', 'VAF', 'ch_status', 'age', 'depth', 'var_depth']], df_hla1[hla_intersect_list]], axis = 1)
df_hla1_scores = pd.concat([df_hla1_hlas, df_hla1_hlas.apply(find_best_score_for_all_variants, df=netmhc_sub2, param=param, axis=1)], axis=1)

In [12]:
# print the df 
# add extra columns (read depth etc + MHC data)
df_hla1_scores_added = pd.concat([df_hla1_scores, df_hla1[['het_allele_I_A', 'het_allele_I_B', 'het_allele_I_C', 'sum_class_I', 'het_all_class_I', 'het_all_class_I_from_allele']]], axis = 1)
df_hla1_scores_added

,Person_ID,gene_var,VAF,ch_status,age,depth,var_depth,A_101,A_102,A_103,...,score_SRSF2_P95L,score_SRSF2_P95R,score_TP53_R175H,score_TP53_R273H,het_allele_I_A,het_allele_I_B,het_allele_I_C,sum_class_I,het_all_class_I,het_all_class_I_from_allele
0,2812213,NaN,NaN,0,41,NaN,NaN,0.0,0.0,0.0,...,3.979,3.656,1.761,1.913,False,True,True,6.0,False,False
1,4860169,NaN,NaN,0,46,NaN,NaN,2.0,0.0,0.0,...,0.973,0.525,2.270,0.132,False,True,True,6.0,False,False
2,3381323,NaN,NaN,0,52,NaN,NaN,0.0,0.0,0.0,...,2.893,1.068,1.468,0.132,True,True,True,6.0,True,True
3,2805252,NaN,NaN,0,65,NaN,NaN,2.0,0.0,0.0,...,1.299,0.420,3.036,0.132,False,True,True,6.0,False,False
4,1118855,NaN,NaN,0,56,NaN,NaN,0.0,0.0,0.0,...,0.751,1.068,1.761,0.132,True,True,True,6.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384910,4478244,DNMT3A_R882H,0.022727,1,55,88.0,2.0,1.0,0.0,0.0,...,0.973,0.420,1.761,0.266,True,True,True,6.0,True,True
384911,5660850,NaN,NaN,0,56,NaN,NaN,1.0,0.0,0.0,...,0.973,0.525,0.995,0.266,True,True,True,6.0,True,True
384912,3573995,NaN,NaN,0,62,NaN,NaN,1.0,0.0,0.0,...,0.973,0.525,1.468,0.266,True,True,True,6.0,True,True
384913,3025735,NaN,NaN,0,51,NaN,NaN,0.0,0.0,0.0,...,0.520,1.068,1.559,0.132,True,True,True,6.0,True,True


In [13]:
# save to a file (this is already usable for further analysis)
df_hla1_scores_added.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_netmhc1_scores_for_all_var.csv')

In [5]:
# TO RUN THE FOLLOWING CELLS TO ADD LABELS, IF YOU HAVE ALREADY SAVED THE DATAFRAMES ABOVE, RUN THIS CELL
df_hla1_scores_added = pd.read_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_netmhc1_scores_for_all_var.csv')

### Add label (top / bottom half of binding scores)

In [14]:
# subset the dataframe to only include gene_var, Person ID and scores 
scores_col = [col for col in df_hla1_scores_added.columns if col.startswith('score_')]
df_hla1_scores_sub = pd.concat([df_hla1_scores_added[['Person_ID', 'gene_var', 'VAF', 'var_depth']], df_hla1_scores_added[scores_col]], axis = 1)

# melt the dataframe 
df_hla1_scores_sub_melted = pd.melt(df_hla1_scores_sub, id_vars = ['Person_ID', 'gene_var', 'VAF', 'var_depth'])

# add a column which has the name of the variant formatted)
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['variable'].str[6:]

# add a column to indicate CH status (either carrier of the variant the score is for, or non-carrier, even if has CH driven by a different variant)
df_hla1_scores_sub_melted['CH_status'] = np.where(df_hla1_scores_sub_melted['gene_var'] == df_hla1_scores_sub_melted['CH_variant'], 1, 0)

# binding score in the format of -1*log10(%Rank_EL)
df_hla1_scores_sub_melted['log_score'] = -1 * np.log10(df_hla1_scores_sub_melted['value'])

# add median score for each variant and then order them by median 
df_hla1_scores_sub_melted['median_score'] = df_hla1_scores_sub_melted.groupby('CH_variant')['log_score'].transform('median')

# convert to categories 
df_hla1_scores_sub_melted['gene_var'] = df_hla1_scores_sub_melted['gene_var'].astype('category') 
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['CH_variant'].astype('category') # make sure you convert this to category first of all

In [15]:
# Add group (split participants with values equal to the median)

# there is quite a lot of people who have the score == median score 
# I am splitting this group so that the number of people with top / bottom is the same: here, each person whose score == median is assigned at random top or bottom 
variants = df_hla1_scores_sub_melted.CH_variant.unique().tolist()

for var in variants:

    # select df with variant 
    df_variant = df_hla1_scores_sub_melted[df_hla1_scores_sub_melted['CH_variant'] == var]
    
    # get the median 
    median_score = df_variant['log_score'].median()
    
    # find median values 
    median_values = df_variant[df_variant['log_score'] == df_variant['median_score']]
    
    # assign top / bottom if above or below median 
    below_median = df_variant[df_variant['log_score'] < median_score]
    above_median = df_variant[df_variant['log_score'] > median_score]
    
    # figure out how many observations you need with median values to top / bottom half
    half_length = len(df_variant) // 2
    num_bottom_needed = half_length - len(below_median)
    num_top_needed = len(median_values) - num_bottom_needed
    
    # assign values equal to median to top or bottom
    shuffled_median_values = median_values.sample(frac=1)
    bottom_half_median = shuffled_median_values.iloc[:num_bottom_needed]
    top_half_median = shuffled_median_values.iloc[num_bottom_needed:]
    
    # assign groups
    df_hla1_scores_sub_melted.loc[below_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[above_median.index, 'group'] = 'top half'
    df_hla1_scores_sub_melted.loc[bottom_half_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[top_half_median.index, 'group'] = 'top half'


In [16]:
# modified saving labels

# save to a file (this is already usable for further analysis)
df_hla1_scores_sub_melted.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_netmhc1_scores_for_all_var_with_labels.csv')


# Load binding predictions for MHC I from PRIME

In [17]:

# Import NetMHC scores  
pred_file_prime = '/Users/barbarawalkowiak/Desktop/msc_thesis/PRIME_out/scores/20240716_PRIME_HLA_UKBB_with_affinities_bestscores_allvariants.csv' # best EL rank, nr of weak and strong binders (I don't have affinity predictions and I am afraid this will have to do)
pred_filename_prime = pred_file_prime.split('/')[2].split('.')[0] # identify file name 
pred_method_prime = pred_file_prime.split('_out')[0] # identify method used to obtain predictions

# load the csv
prime = pd.read_csv(pred_file_prime) # load the csv 
prime = prime.rename(columns={'allele': 'HLA'})

# specify HLA allele naming format 
prime_col = prime.HLA
prime_formatted = prime_col.apply(transform_format_prime)
prime = pd.concat([prime, prime_formatted.rename('HLA_formatted')], axis = 1)

# Replace STOP with *
prime['variant'] = prime['variant'].str.replace('STOP', '*')

# rename column from min_rank to '%Rank_EL' to keep it consistent with NetMHC
prime = prime.rename(columns={'min_rank': '%Rank_EL'})

# select required columns and sor values 
prime = prime[['HLA_formatted', '%Rank_EL', 'sum_peptides_below_05', 'sum_peptides_below_2', 'gene', 'variant', 'genotype']]
prime = prime.sort_values(by=['HLA_formatted', 'gene', 'variant', 'genotype'])

# identify gene variants 
prime['gene_var_gt'] = prime['gene'] + '_' + prime['variant'] + '_' + prime['genotype'] # add complete genotype data
prime['gene_var'] = prime['gene'] + '_' + prime['variant']
prime['varID'] = prime['gene'] + ' ' + prime['variant'] # this is a column where the variant ID is in the same format as in the CH cases dataframe
scores_prime = prime[['HLA_formatted', '%Rank_EL', 'varID', 'gene_var', 'gene_var_gt']] # select columns of interest (realistically I only want minimum %EL rank anyway)

# Print the results 
print('Number of alleles for which predictions are available (NetMHC):', len(prime.HLA_formatted.unique()))

# Find MHC alleles which have been typed in the UKBB
hla_ukbb = df_hla1.filter(regex='\d').columns # HLA from all UKBB
print('Number of HLA alleles which have been identified in the UK BioBank:', len(hla_ukbb))

# Now look at variants
print('Number of unique variants I have predictions for (NetMHC):',  len(prime.gene_var.unique()))

Number of alleles for which predictions are available (NetMHC): 194
Number of HLA alleles which have been identified in the UK BioBank: 215
Number of unique variants I have predictions for (NetMHC): 54


In [18]:
print('Number of variants identified in the UKBB from CH screening:', len(df_hla1.gene_var.unique())-1) # NB this list also includes 'NaN' that's why its -1
print('Number or variants I have predictions for (PRIME):',  len(prime.gene_var.unique()))

variants_in_ukbb = df_hla1.gene_var.unique().tolist()
variants_in_ukbb = [x for x in variants_in_ukbb if str(x) != 'nan']
print('List of variants identified in UKBB:', sorted(variants_in_ukbb))
print('List of variants with PRIME preds:',  sorted(prime.gene_var.unique())) # there is more because I also made predictions for variants which ended up not being common enough 

Number of variants identified in the UKBB from CH screening: 40
Number or variants I have predictions for (PRIME): 54
List of variants identified in UKBB: ['DNMT3A_P904L', 'DNMT3A_P904Q', 'DNMT3A_P904R', 'DNMT3A_R320*', 'DNMT3A_R326C', 'DNMT3A_R326G', 'DNMT3A_R326S', 'DNMT3A_R598*', 'DNMT3A_R729G', 'DNMT3A_R729W', 'DNMT3A_R736C', 'DNMT3A_R736G', 'DNMT3A_R736H', 'DNMT3A_R736L', 'DNMT3A_R736S', 'DNMT3A_R771*', 'DNMT3A_R882C', 'DNMT3A_R882H', 'DNMT3A_R882L', 'DNMT3A_R882P', 'DNMT3A_R882S', 'DNMT3A_Y735C', 'DNMT3A_Y735F', 'DNMT3A_Y735S', 'GNB1_K57E', 'IDH1_R132H', 'IDH2_R140Q', 'IDH2_R172K', 'JAK2_V617F', 'KRAS_G12D', 'KRAS_G12S', 'MPL_W515L', 'NRAS_G12D', 'SF3B1_K666N', 'SF3B1_K700E', 'SRSF2_P95H', 'SRSF2_P95L', 'SRSF2_P95R', 'TP53_R175H', 'TP53_R273H']
List of variants with PRIME preds: ['DNMT3A_P904L', 'DNMT3A_P904Q', 'DNMT3A_P904R', 'DNMT3A_R320*', 'DNMT3A_R326C', 'DNMT3A_R326G', 'DNMT3A_R326S', 'DNMT3A_R598*', 'DNMT3A_R598G', 'DNMT3A_R729G', 'DNMT3A_R729W', 'DNMT3A_R736C', 'DNMT3A_R73

In [19]:

# create a suitably formatted NetMHC df 
param = '%Rank_EL'
prime_sub = prime[['HLA_formatted', 'gene_var_gt', param]]
prime_sub_wide = pd.pivot(prime_sub, index='gene_var_gt', columns='HLA_formatted', values=param)
prime_sub_wide = prime_sub_wide.reset_index() # this is to make sure that you have the gene_var column in there too 
prime_sub_wide.columns = map(transform_format_prime, prime_sub_wide.columns)
prime_sub_wide

# it makes sense that we have 104 rows bc 2 * 54 variants 

,gene_var_gt,A_101,A_102,A_103,A_1101,A_1102,A_1103,A_201,A_202,A_203,...,C_403,C_407,C_501,C_602,C_701,C_702,C_704,C_801,C_802,C_804
0,DNMT3A_P904L_ch,1.490,1.490,1.490,1.167,1.210,1.167,0.105,0.034,0.019,...,0.319,0.100,0.560,0.231,0.461,0.192,0.484,0.250,0.976,0.976
1,DNMT3A_P904L_refseq,2.985,2.985,2.985,1.796,1.846,1.796,1.138,0.265,0.172,...,0.255,0.080,0.706,0.513,1.678,0.455,1.026,0.259,1.428,1.428
2,DNMT3A_P904Q_ch,2.544,2.544,2.544,3.406,2.601,3.406,0.093,0.040,0.019,...,1.075,0.434,1.044,0.082,1.299,0.440,1.945,0.311,1.475,1.475
3,DNMT3A_P904Q_refseq,2.985,2.985,2.985,1.796,1.846,1.796,1.138,0.265,0.172,...,0.255,0.080,0.706,0.513,1.678,0.455,1.026,0.259,1.428,1.428
4,DNMT3A_P904R_ch,2.846,2.846,2.846,0.688,0.756,0.688,0.073,0.031,0.018,...,0.798,0.609,0.922,0.049,0.719,0.469,1.726,0.514,1.859,1.859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,SRSF2_P95R_refseq,7.313,7.313,7.313,4.763,7.841,4.763,11.579,11.259,12.816,...,5.217,5.758,5.444,1.548,1.109,0.456,1.007,1.956,8.023,8.023
104,TP53_R175H_ch,2.672,2.672,2.672,4.340,5.084,4.340,0.501,0.250,0.555,...,4.220,3.669,5.319,0.740,1.556,0.401,0.795,3.628,8.944,8.944
105,TP53_R175H_refseq,4.638,4.638,4.638,1.639,1.989,1.639,1.332,0.625,1.604,...,10.332,8.958,13.608,1.888,0.741,0.595,2.867,13.566,18.073,18.073
106,TP53_R273H_ch,5.004,5.004,5.004,5.790,4.049,5.790,0.764,0.717,1.059,...,0.806,1.050,2.372,0.029,0.073,0.260,0.078,1.387,3.657,3.657


In [20]:

# create a list of HLA alleles genotyped in the UKBB for which predictions are available
hla_intersect = prime_sub_wide.columns[prime_sub_wide.columns.isin(hla_ukbb)] # HLA in the UKBB which I have predictions for 
hla_intersect_list = hla_intersect.tolist() # 194 alleles 

# create the prediction dataset 
prime_sub = prime_sub_wide[hla_intersect_list + prime_sub_wide.columns[prime_sub_wide.columns.str.contains('gene_var')].tolist()] # subset netmhc so you only have alleles which are in the UKBB, also 194
prime_sub = prime_sub[prime_sub['gene_var_gt'].str.contains('_ch', regex=True)] # retain CH scores only 
prime_sub['gene_var'] = prime_sub['gene_var_gt'].str.replace('_ch', '') # remove the ch / refseq annotation
prime_sub['gene_var'] = prime_sub['gene_var'].str.replace('_refseq', '') # remove refseq if present 
prime_sub = prime_sub[prime_sub['gene_var'].isin(variants_in_ukbb)]

In [21]:
# apply the function to find, for each participant, the best score for each examined CH variant 
# you take your current df and apply, row by row, the function to get scores for each variant 
# for the purpose of this, we first need to select MHC columns which are actually in the hla_intersect_list I think 
# DO NOT use this for sth like MHC heterozygosity tho!!
df_hla1_hlas = pd.concat([df_hla1[['Person_ID', 'gene_var', 'VAF', 'ch_status', 'age', 'depth', 'var_depth']], df_hla1[hla_intersect_list]], axis = 1)
df_hla1_scores = pd.concat([df_hla1_hlas, df_hla1_hlas.apply(find_best_score_for_all_variants, df=prime_sub, param=param, axis=1)], axis=1)

In [22]:
# print the df (just check it looks okay)
# add extra columns (read depth etc + MHC data)
df_hla1_scores_added = pd.concat([df_hla1_scores, df_hla1[['het_allele_I_A', 'het_allele_I_B', 'het_allele_I_C', 'sum_class_I', 'het_all_class_I', 'het_all_class_I_from_allele']]], axis = 1)
df_hla1_scores_added

,Person_ID,gene_var,VAF,ch_status,age,depth,var_depth,A_101,A_102,A_103,...,score_TP53_R175H,score_TP53_R273H,het_allele_I_A,het_allele_I_B,het_allele_I_C,sum_class_I,het_all_class_I,het_all_class_I_from_allele,depth,var_depth
0,2812213,NaN,NaN,0,41,NaN,NaN,0.0,0.0,0.0,...,0.501,0.764,False,True,True,6.0,False,False,NaN,NaN
1,4860169,NaN,NaN,0,46,NaN,NaN,2.0,0.0,0.0,...,0.740,0.029,False,True,True,6.0,False,False,NaN,NaN
2,3381323,NaN,NaN,0,52,NaN,NaN,0.0,0.0,0.0,...,0.385,0.029,True,True,True,6.0,True,True,NaN,NaN
3,2805252,NaN,NaN,0,65,NaN,NaN,2.0,0.0,0.0,...,0.401,0.029,False,True,True,6.0,False,False,NaN,NaN
4,1118855,NaN,NaN,0,56,NaN,NaN,0.0,0.0,0.0,...,0.501,0.029,True,True,True,6.0,True,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384910,4478244,DNMT3A_R882H,0.022727,1,55,88.0,2.0,1.0,0.0,0.0,...,0.401,0.073,True,True,True,6.0,True,True,88.0,2.0
384911,5660850,NaN,NaN,0,56,NaN,NaN,1.0,0.0,0.0,...,0.281,0.073,True,True,True,6.0,True,True,NaN,NaN
384912,3573995,NaN,NaN,0,62,NaN,NaN,1.0,0.0,0.0,...,0.385,0.073,True,True,True,6.0,True,True,NaN,NaN
384913,3025735,NaN,NaN,0,51,NaN,NaN,0.0,0.0,0.0,...,0.740,0.029,True,True,True,6.0,True,True,NaN,NaN


In [23]:
# save to a file (this is already usable for further analysis)
df_hla1_scores_added.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_prime_scores_for_all_var.csv')


In [56]:
# FOR LABELS, READ THIS IN (in case kernel crashes etc.)
df_hla1_scores_added = pd.read_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_prime_scores_for_all_var.csv')

## Add labels (top / bottom half of binding scores)

In [29]:
# add median scores

# subset the dataframe to only include gene_var, Person ID and scores 
scores_col = [col for col in df_hla1_scores_added_dp.columns if col.startswith('score_')]
df_hla1_scores_sub = pd.concat([df_hla1_scores_added_dp[['Person_ID', 'gene_var', 'VAF', 'var_depth']], df_hla1_scores_added_dp[scores_col]], axis = 1)

# melt the dataframe 
df_hla1_scores_sub_melted = pd.melt(df_hla1_scores_sub, id_vars = ['gene_var', 'Person_ID', 'VAF', 'var_depth'])

# add a column which has the name of the variant (nicely formatted)
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['variable'].str[6:]

# add a column to indicate CH status (either carrier of the variant the score is for, or non-carrier, even if have CH driven by sth else)
df_hla1_scores_sub_melted['CH_status'] = np.where(df_hla1_scores_sub_melted['gene_var'] == df_hla1_scores_sub_melted['CH_variant'].replace(), 1, 0)

# you want the %Rank_EL to be in the format of -1*log10(%Rank_EL)
df_hla1_scores_sub_melted['log_score'] = -1 * np.log10(df_hla1_scores_sub_melted['value'])

# add median score for each variant and then order them by median 
df_hla1_scores_sub_melted['median_score'] = df_hla1_scores_sub_melted.groupby('CH_variant')['log_score'].transform('median')

# convert to categories 
df_hla1_scores_sub_melted['gene_var'] = df_hla1_scores_sub_melted['gene_var'].astype('category') 
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['CH_variant'].astype('category') # make sure you convert this to category first of all

In [30]:
# Add group (split participants with values equal to the median)

# there is quite a lot of people who have the score == median score 
# I am splitting this group so that the number of people with top / bottom is the same: here, each person whose score == median is assigned at random top or bottom 
variants = df_hla1_scores_sub_melted.CH_variant.unique().tolist()

for var in variants:

    # select df with variant 
    df_variant = df_hla1_scores_sub_melted[df_hla1_scores_sub_melted['CH_variant'] == var]
    
    # get the median 
    median_score = df_variant['log_score'].median()
    
    # find median values 
    median_values = df_variant[df_variant['log_score'] == df_variant['median_score']]
    
    # assign top / bottom if above or below median 
    below_median = df_variant[df_variant['log_score'] < median_score]
    above_median = df_variant[df_variant['log_score'] > median_score]
    
    # figure out how many observations you need with median values to top / bottom half
    half_length = len(df_variant) // 2
    num_bottom_needed = half_length - len(below_median)
    num_top_needed = len(median_values) - num_bottom_needed
    
    # assign values equal to median to top or bottom
    shuffled_median_values = median_values.sample(frac=1)
    bottom_half_median = shuffled_median_values.iloc[:num_bottom_needed]
    top_half_median = shuffled_median_values.iloc[num_bottom_needed:]
    
    # assign groups
    df_hla1_scores_sub_melted.loc[below_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[above_median.index, 'group'] = 'top half'
    df_hla1_scores_sub_melted.loc[bottom_half_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[top_half_median.index, 'group'] = 'top half'


In [31]:
# save to csv file
# save to a file (this is already usable for further analysis)
df_hla1_scores_sub_melted.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/dataframes/20240907_prime_scores_for_all_var_with_labels.csv')

: 

# Alternative scores

- Binding affinity 

- DAI (based on %EL rank)

- DAI (based on binding affinity)

In [26]:
# 1 get binding affinities 

# create a suitably formatted NetMHC df 
param = 'Aff_nM'
netmhc_sub = netmhc[['HLA_formatted', 'gene_var_gt', param]]
netmhc_sub_wide = pd.pivot(netmhc_sub, index='gene_var_gt', columns='HLA_formatted', values=param)
netmhc_sub_wide = netmhc_sub_wide.reset_index() # this is to make sure that you have the gene_var column in there too 
netmhc_sub_wide.columns = map(transform_format_netmhc, netmhc_sub_wide.columns)

# create a list of HLA alleles genotyped in the UKBB for which predictions are available
hla_intersect = netmhc_sub_wide.columns[netmhc_sub_wide.columns.isin(hla_ukbb)] # HLA in the UKBB which I have predictions for 
hla_intersect_list = hla_intersect.tolist() # 194 alleles 

# create the prediction dataset 
netmhc_sub = netmhc_sub_wide[hla_intersect_list + netmhc_sub_wide.columns[netmhc_sub_wide.columns.str.contains('gene_var')].tolist()] # subset netmhc so you only have alleles which are in the UKBB, also 194
netmhc_sub = netmhc_sub[netmhc_sub['gene_var_gt'].str.contains('_ch', regex=True)] # retain CH scores only 
netmhc_sub['gene_var'] = netmhc_sub['gene_var_gt'].str.replace('_ch', '') # remove the ch / refseq annotation
netmhc_sub['gene_var'] = netmhc_sub['gene_var'].str.replace('_refseq', '') # remove refseq if present 

# apply the function to find, for each participant, the best score for each examined CH variant 
# you take your current df and apply, row by row, the function to get scores for each variant 
# for the purpose of this, we first need to select MHC columns which are actually in the hla_intersect_list I think 
# DO NOT use this for sth like MHC heterozygosity tho!!
df_hla1_hlas = pd.concat([df_hla1[['Person_ID', 'gene_var', 'VAF', 'ch_status', 'age']], df_hla1[hla_intersect_list]], axis = 1)
df_hla1_scores = pd.concat([df_hla1_hlas, df_hla1_hlas.apply(find_best_score_for_all_variants, df=netmhc_sub, param=param, axis=1)], axis=1)

# print the df (just check it looks okay)
# add extra columns (read depth etc + MHC data)
df_hla1_scores_added = pd.concat([df_hla1_scores, df_hla1[['het_allele_I_A', 'het_allele_I_B', 'het_allele_I_C', 'sum_class_I', 'het_all_class_I', 'het_all_class_I_from_allele', 'depth', 'var_depth']]], axis = 1)

# save to a file (this is already usable for further analysis)
df_hla1_scores_added.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/20240820_netmhc1_scores_for_all_var_bindaff.csv')

NameError: name 'netmhc' is not defined

In [12]:
# add labels 

# add median scores

# subset the dataframe to only include gene_var, Person ID and scores 
scores_col = [col for col in df_hla1_scores_added.columns if col.startswith('score_')]
df_hla1_scores_sub = pd.concat([df_hla1_scores_added[['Person_ID', 'gene_var']], df_hla1_scores_added[scores_col]], axis = 1)

# melt the dataframe 
df_hla1_scores_sub_melted = pd.melt(df_hla1_scores_sub, id_vars = ['gene_var', 'Person_ID'])

# add a column which has the name of the variant (nicely formatted)
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['variable'].str[6:]

# add a column to indicate CH status (either carrier of the variant the score is for, or non-carrier, even if have CH driven by sth else)
df_hla1_scores_sub_melted['CH_status'] = np.where(df_hla1_scores_sub_melted['gene_var'] == df_hla1_scores_sub_melted['CH_variant'].replace(), 'carrier', 'non-carrier')

# you want the %Rank_EL to be in the format of -1*log10(%Rank_EL)
df_hla1_scores_sub_melted['log_score'] = -1 * np.log10(df_hla1_scores_sub_melted['value'])

# add median score for each variant and then order them by median 
df_hla1_scores_sub_melted['median_score'] = df_hla1_scores_sub_melted.groupby('CH_variant')['log_score'].transform('median')

# convert to categories 
df_hla1_scores_sub_melted['gene_var'] = df_hla1_scores_sub_melted['gene_var'].astype('category') 
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['CH_variant'].astype('category')

variants = df_hla1_scores_sub_melted.CH_variant.unique().tolist()

for var in variants:

    # select df with variant 
    df_variant = df_hla1_scores_sub_melted[df_hla1_scores_sub_melted['CH_variant'] == var]
    
    # get the median 
    median_score = df_variant['log_score'].median()
    
    # find median values 
    median_values = df_variant[df_variant['log_score'] == df_variant['median_score']]
    
    # assign top / bottom if above or below median 
    below_median = df_variant[df_variant['log_score'] < median_score]
    above_median = df_variant[df_variant['log_score'] > median_score]
    
    # figure out how many observations you need with median values to top / bottom half
    half_length = len(df_variant) // 2
    num_bottom_needed = half_length - len(below_median)
    num_top_needed = len(median_values) - num_bottom_needed
    
    # assign values equal to median to top or bottom
    shuffled_median_values = median_values.sample(frac=1)
    bottom_half_median = shuffled_median_values.iloc[:num_bottom_needed]
    top_half_median = shuffled_median_values.iloc[num_bottom_needed:]
    
    # assign groups
    df_hla1_scores_sub_melted.loc[below_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[above_median.index, 'group'] = 'top half'
    df_hla1_scores_sub_melted.loc[bottom_half_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[top_half_median.index, 'group'] = 'top half'

df_hla1_scores_sub_melted.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/20240820_netmhc1_scores_for_all_var_with_labels_bindaff.csv')


In [13]:
# get DAI (based on %EL Rank)

# DAIBA=WTBA-mutBA and DAIR=mutR/WTR

netmhc['dai_rank_el'] = netmhc['%Rank_EL'] / netmhc['%Rank_EL'].shift(-1) # mutant over wt
netmhc['dai_rank_aff'] = netmhc['Aff_nM'].shift(-1) - netmhc['Aff_nM']
netmhc2 = netmhc.iloc[0::2].reset_index(drop=True)


In [14]:
# need to do something to assign, for everyone, the score for every variant we have predictions for based on their MHC I

# define the function to find best scores (we can technically do this for different parameters, realistically I think will just do it for %Rank_EL)

def find_best_score_for_all_variants(row, df, param):

    '''
    row = function is applied to each row of the participant dataframe (ie run through each participant)

    df = dataframe with prediction scores (like netmhc)

    Allowed parameters (param) are:
    Aff_nM - affinity (raw number)
    Score_BA - binding affinity score
    Score_EL - elution score
    %Rank_BA - %Rank of binding affinity cf a set of random peptides
    %Rank_EL - %Rank of elution cf a set of random peptides
    '''


    # get HLAs for each person 
    hlas = row.index[5:][row[5:] >= 1] # select alleles which each Person (row) carries (the first 5 columns are: Person ID, gene_var, VAF, CH status, age)
   
    # get variants 
    variants = df['gene_var']
   
    scores = {} # initialise empy dictionaries

    # depending on the parameter, pick the minimum of maximum value 
    if param == "Aff_nM":
        for var in variants:
            # Find the minimum value for each variant in the category that is present
            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0])
            # Update the dictionary with the minimum value for the corresponding variant
            scores[f'score_{var}'] = best_value
        return pd.Series(scores)

    elif param == "Score_BA":
        for var in variants:
            
            best_value = max(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value
        
        return pd.Series(scores)

    elif param == "Score_EL":
        for var in variants:
           
            best_value = max(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value

        return pd.Series(scores)

    elif param == "%Rank_BA":
        for var in variants:

            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value

        return pd.Series(scores)

    # we will likely be choosing this option
    elif param == "%Rank_EL":
        for var in variants:
            
            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0]) # choose minimum rank as the best score 
            scores[f'score_{var}'] = best_value

        return pd.Series(scores)

    elif param == "dai_rank_el": # mutant over wt but we have small numbers so higher is better 
        for var in variants:
            
            best_value = min(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value
        
        return pd.Series(scores)

    elif param == "dai_rank_aff": # wt - mutant so higher is better 
        for var in variants:
            
            best_value = max(df.loc[df['gene_var'] == var, hlas].values[0])
            scores[f'score_{var}'] = best_value
        
        return pd.Series(scores)

In [55]:

param = 'dai_rank_el'
netmhc_sub = netmhc2[['HLA_formatted', 'gene_var_gt', param]]
netmhc_sub_wide = pd.pivot(netmhc_sub, index='gene_var_gt', columns='HLA_formatted', values=param)
netmhc_sub_wide = netmhc_sub_wide.reset_index() # this is to make sure that you have the gene_var column in there too 
netmhc_sub_wide.columns = map(transform_format_netmhc, netmhc_sub_wide.columns)

# create a list of HLA alleles genotyped in the UKBB for which predictions are available
hla_intersect = netmhc_sub_wide.columns[netmhc_sub_wide.columns.isin(hla_ukbb)] # HLA in the UKBB which I have predictions for 
hla_intersect_list = hla_intersect.tolist() # 194 alleles 

# create the prediction dataset 
netmhc_sub = netmhc_sub_wide[hla_intersect_list + netmhc_sub_wide.columns[netmhc_sub_wide.columns.str.contains('gene_var')].tolist()] # subset netmhc so you only have alleles which are in the UKBB, also 194
netmhc_sub = netmhc_sub[netmhc_sub['gene_var_gt'].str.contains('_ch', regex=True)] # retain CH scores only 
netmhc_sub['gene_var'] = netmhc_sub['gene_var_gt'].str.replace('_ch', '') # remove the ch / refseq annotation
netmhc_sub['gene_var'] = netmhc_sub['gene_var'].str.replace('_refseq', '') # remove refseq if present 

# apply the function to find, for each participant, the best score for each examined CH variant 
# you take your current df and apply, row by row, the function to get scores for each variant 
# for the purpose of this, we first need to select MHC columns which are actually in the hla_intersect_list I think 
# DO NOT use this for sth like MHC heterozygosity tho!!
df_hla1_hlas = pd.concat([df_hla1[['Person_ID', 'gene_var', 'VAF', 'ch_status', 'age']], df_hla1[hla_intersect_list]], axis = 1)
df_hla1_scores = pd.concat([df_hla1_hlas, df_hla1_hlas.apply(find_best_score_for_all_variants, df=netmhc_sub, param=param, axis=1)], axis=1)

# print the df (just check it looks okay)
# add extra columns (read depth etc + MHC data)
df_hla1_scores_added = pd.concat([df_hla1_scores, df_hla1[['het_allele_I_A', 'het_allele_I_B', 'het_allele_I_C', 'sum_class_I', 'het_all_class_I', 'het_all_class_I_from_allele', 'depth', 'var_depth']]], axis = 1)

# save to a file (this is already usable for further analysis)
df_hla1_scores_added.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/20240820_netmhc1_scores_for_all_var_dai_rank_el.csv')


In [56]:
# add labels 

# add median scores

# subset the dataframe to only include gene_var, Person ID and scores 
scores_col = [col for col in df_hla1_scores_added.columns if col.startswith('score_')]
df_hla1_scores_sub = pd.concat([df_hla1_scores_added[['Person_ID', 'gene_var']], df_hla1_scores_added[scores_col]], axis = 1)

# melt the dataframe 
df_hla1_scores_sub_melted = pd.melt(df_hla1_scores_sub, id_vars = ['gene_var', 'Person_ID'])

# add a column which has the name of the variant (nicely formatted)
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['variable'].str[6:]

# add a column to indicate CH status (either carrier of the variant the score is for, or non-carrier, even if have CH driven by sth else)
df_hla1_scores_sub_melted['CH_status'] = np.where(df_hla1_scores_sub_melted['gene_var'] == df_hla1_scores_sub_melted['CH_variant'].replace(), 'carrier', 'non-carrier')

# you want the %Rank_EL to be in the format of -1*log10(%Rank_EL)
df_hla1_scores_sub_melted['log_score'] = -1 * np.log10(df_hla1_scores_sub_melted['value'])

# add median score for each variant and then order them by median 
df_hla1_scores_sub_melted['median_score'] = df_hla1_scores_sub_melted.groupby('CH_variant')['log_score'].transform('median')

# convert to categories 
df_hla1_scores_sub_melted['gene_var'] = df_hla1_scores_sub_melted['gene_var'].astype('category') 
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['CH_variant'].astype('category')

variants = df_hla1_scores_sub_melted.CH_variant.unique().tolist()

for var in variants:

    # select df with variant 
    df_variant = df_hla1_scores_sub_melted[df_hla1_scores_sub_melted['CH_variant'] == var]
    
    # get the median 
    median_score = df_variant['log_score'].median()
    
    # find median values 
    median_values = df_variant[df_variant['log_score'] == df_variant['median_score']]
    
    # assign top / bottom if above or below median 
    below_median = df_variant[df_variant['log_score'] < median_score]
    above_median = df_variant[df_variant['log_score'] > median_score]
    
    # figure out how many observations you need with median values to top / bottom half
    half_length = len(df_variant) // 2
    num_bottom_needed = half_length - len(below_median)
    num_top_needed = len(median_values) - num_bottom_needed
    
    # assign values equal to median to top or bottom
    shuffled_median_values = median_values.sample(frac=1)
    bottom_half_median = shuffled_median_values.iloc[:num_bottom_needed]
    top_half_median = shuffled_median_values.iloc[num_bottom_needed:]
    
    # assign groups
    df_hla1_scores_sub_melted.loc[below_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[above_median.index, 'group'] = 'top half'
    df_hla1_scores_sub_melted.loc[bottom_half_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[top_half_median.index, 'group'] = 'top half'

df_hla1_scores_sub_melted.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/20240820_netmhc1_scores_for_all_var_with_labels_dai_rank_el.csv')


In [15]:
param = 'dai_rank_aff'
netmhc_sub = netmhc2[['HLA_formatted', 'gene_var_gt', param]]
netmhc_sub_wide = pd.pivot(netmhc_sub, index='gene_var_gt', columns='HLA_formatted', values=param)
netmhc_sub_wide = netmhc_sub_wide.reset_index() # this is to make sure that you have the gene_var column in there too 
netmhc_sub_wide.columns = map(transform_format_netmhc, netmhc_sub_wide.columns)

# create a list of HLA alleles genotyped in the UKBB for which predictions are available
hla_intersect = netmhc_sub_wide.columns[netmhc_sub_wide.columns.isin(hla_ukbb)] # HLA in the UKBB which I have predictions for 
hla_intersect_list = hla_intersect.tolist() # 194 alleles 

# create the prediction dataset 
netmhc_sub = netmhc_sub_wide[hla_intersect_list + netmhc_sub_wide.columns[netmhc_sub_wide.columns.str.contains('gene_var')].tolist()] # subset netmhc so you only have alleles which are in the UKBB, also 194
netmhc_sub = netmhc_sub[netmhc_sub['gene_var_gt'].str.contains('_ch', regex=True)] # retain CH scores only 
netmhc_sub['gene_var'] = netmhc_sub['gene_var_gt'].str.replace('_ch', '') # remove the ch / refseq annotation
netmhc_sub['gene_var'] = netmhc_sub['gene_var'].str.replace('_refseq', '') # remove refseq if present 

# apply the function to find, for each participant, the best score for each examined CH variant 
# you take your current df and apply, row by row, the function to get scores for each variant 
# for the purpose of this, we first need to select MHC columns which are actually in the hla_intersect_list I think 
# DO NOT use this for sth like MHC heterozygosity tho!!
df_hla1_hlas = pd.concat([df_hla1[['Person_ID', 'gene_var', 'VAF', 'ch_status', 'age']], df_hla1[hla_intersect_list]], axis = 1)
df_hla1_scores = pd.concat([df_hla1_hlas, df_hla1_hlas.apply(find_best_score_for_all_variants, df=netmhc_sub, param=param, axis=1)], axis=1)

# print the df (just check it looks okay)
# add extra columns (read depth etc + MHC data)
df_hla1_scores_added = pd.concat([df_hla1_scores, df_hla1[['het_allele_I_A', 'het_allele_I_B', 'het_allele_I_C', 'sum_class_I', 'het_all_class_I', 'het_all_class_I_from_allele', 'depth', 'var_depth']]], axis = 1)

# save to a file (this is already usable for further analysis)
df_hla1_scores_added.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/20240820_netmhc1_scores_for_all_var_dai_aff.csv')

In [20]:
# add labels 
# add labels 

# add median scores

# subset the dataframe to only include gene_var, Person ID and scores 
scores_col = [col for col in df_hla1_scores_added.columns if col.startswith('score_')]
df_hla1_scores_sub = pd.concat([df_hla1_scores_added[['Person_ID', 'gene_var']], df_hla1_scores_added[scores_col]], axis = 1)

# melt the dataframe 
df_hla1_scores_sub_melted = pd.melt(df_hla1_scores_sub, id_vars = ['gene_var', 'Person_ID'])

# add a column which has the name of the variant (nicely formatted)
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['variable'].str[6:]

# add a column to indicate CH status (either carrier of the variant the score is for, or non-carrier, even if have CH driven by sth else)
df_hla1_scores_sub_melted['CH_status'] = np.where(df_hla1_scores_sub_melted['gene_var'] == df_hla1_scores_sub_melted['CH_variant'].replace(), 'carrier', 'non-carrier')

# NOTE: not doing log score here (because I want to know if the difference is positive / negative)
# add median score for each variant and then order them by median 
df_hla1_scores_sub_melted['median_score'] = df_hla1_scores_sub_melted.groupby('CH_variant')['value'].transform('median')

# convert to categories 
df_hla1_scores_sub_melted['gene_var'] = df_hla1_scores_sub_melted['gene_var'].astype('category') 
df_hla1_scores_sub_melted['CH_variant'] = df_hla1_scores_sub_melted['CH_variant'].astype('category')

variants = df_hla1_scores_sub_melted.CH_variant.unique().tolist()

for var in variants:

    # select df with variant 
    df_variant = df_hla1_scores_sub_melted[df_hla1_scores_sub_melted['CH_variant'] == var]
    
    # get the median 
    median_score = df_variant['value'].median()
    
    # find median values 
    median_values = df_variant[df_variant['value'] == df_variant['median_score']]
    
    # assign top / bottom if above or below median 
    below_median = df_variant[df_variant['value'] < median_score]
    above_median = df_variant[df_variant['value'] > median_score]
    
    # figure out how many observations you need with median values to top / bottom half
    half_length = len(df_variant) // 2
    num_bottom_needed = half_length - len(below_median)
    num_top_needed = len(median_values) - num_bottom_needed
    
    # assign values equal to median to top or bottom
    shuffled_median_values = median_values.sample(frac=1)
    bottom_half_median = shuffled_median_values.iloc[:num_bottom_needed]
    top_half_median = shuffled_median_values.iloc[num_bottom_needed:]
    
    # assign groups
    df_hla1_scores_sub_melted.loc[below_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[above_median.index, 'group'] = 'top half'
    df_hla1_scores_sub_melted.loc[bottom_half_median.index, 'group'] = 'bottom half'
    df_hla1_scores_sub_melted.loc[top_half_median.index, 'group'] = 'top half'

df_hla1_scores_sub_melted.to_csv('/Users/barbarawalkowiak/Desktop/msc_thesis/results/20240820_netmhc1_scores_for_all_var_with_labels_dai_rank_aff.csv')

: 